In [33]:
from eia_api_client import EIAClient, Frequency, DataType, Facet
import json
from datetime import datetime, timezone, timedelta

In [34]:
with open("config.json", "r") as f:
    config_json = json.load(f)

api_key = config_json.get("api_key")

eia_client = EIAClient(api_key)
eia_response = eia_client.get_data(
    frequency=Frequency.LOCAL_HOURLY,
    data_type=DataType.VALUE,
    facet=Facet.MIDA,
    start=datetime(2023, 8, 1, tzinfo=timezone(offset=-timedelta(hours=4))),
    end=datetime(2023, 8, 31, tzinfo=timezone(offset=-timedelta(hours=4)))
)

In [35]:
data = eia_response.data
data[:10]

[Data(period='2023-07-31T20-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=115248, value_units='megawatthours'),
 Data(period='2023-07-31T21-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=110343, value_units='megawatthours'),
 Data(period='2023-07-31T22-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=105699, value_units='megawatthours'),
 Data(period='2023-07-31T23-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=99422, value_units='megawatthours'),
 Data(period='2023-08-01T00-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=93557, value_units='megawatthours'),
 Data(period='2023-08-01T01-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=88047, value_units='megawatthours'),
 Data(period='2023-08-01T02-04', respondent='MIDA', res

In [36]:
demand_data = list(filter(lambda x: x.type == "D", data))
demand_data[:10]

[Data(period='2023-07-31T20-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=115248, value_units='megawatthours'),
 Data(period='2023-07-31T21-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=110343, value_units='megawatthours'),
 Data(period='2023-07-31T22-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=105699, value_units='megawatthours'),
 Data(period='2023-07-31T23-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=99422, value_units='megawatthours'),
 Data(period='2023-08-01T00-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=93557, value_units='megawatthours'),
 Data(period='2023-08-01T01-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=88047, value_units='megawatthours'),
 Data(period='2023-08-01T02-04', respondent='MIDA', res

In [37]:
import plotly.express as px

fig = px.bar(x=[data_item.period for data_item in demand_data], y=[data_item.value for data_item in demand_data])
fig.show()

In [38]:
[data_item for data_item in data if data_item.period == "2023-06-24T05-04"]

[]

In [41]:
recent_demand_data = list(filter(lambda d: "2023-08-30" in d.period, demand_data))
recent_demand_data[:5]

[Data(period='2023-08-30T00-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=95073, value_units='megawatthours'),
 Data(period='2023-08-30T01-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=90813, value_units='megawatthours'),
 Data(period='2023-08-30T02-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=87689, value_units='megawatthours'),
 Data(period='2023-08-30T03-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=85166, value_units='megawatthours'),
 Data(period='2023-08-30T04-04', respondent='MIDA', respondent_name='Mid-Atlantic', type='D', type_name='Demand', value=83846, value_units='megawatthours')]

In [47]:
fig = px.bar(x=[data_item.period_datetime for data_item in recent_demand_data], y=[data_item.value for data_item in recent_demand_data])
fig.update_layout(
    xaxis_title="period",
    yaxis_title="megawatt hours",
    title="Mid Atlantic Energy Demand"
)
fig.show()

In [ ]:
from eia_api_client import Data

def get_charge_start_time(charge_time: int, data: list[Data]) -> datetime:
    charge_time_start_index = None
    for i in range(0, len(data) - charge_time + 1):
        sum_demand = sum([data_item.value for data_item in data[i:i + charge_time]])
        if not charge_time_start_index:
            charge_time_start_index = (i, sum_demand)
        elif sum_demand < charge_time_start_index[1]:
            charge_time_start_index = (i, sum_demand)
    return data[charge_time_start_index[0]].period_datetime

start_charge_time = get_charge_start_time(3, recent_demand_data)
str(start_charge_time)

In [ ]:
highest_energy_hour = max(recent_demand_data, key=lambda data: data.value)
highest_energy_hour

In [ ]:
total_megawatt_hours = sum([data.value for data in recent_demand_data])
total_megawatt_hours

In [ ]:
peak_hour_energy_percentage = highest_energy_hour.value / total_megawatt_hours
peak_hour_energy_percentage

In [ ]:
def find_peak_hour_timeframe(demand_data):
    """
    Given demand data from the EIA API
    Find the consecutive hour blocks that constitute roughly 20% of daily energy demand
    i.e. 2pm - 6pm for the East Coast
    """
    highest_energy_hour = max(demand_data, key=lambda data: data.value)
    current_index = demand_data.index(highest_energy_hour)
    left_index = current_index - 1
    right_index = current_index + 1
    peak_hours_percentage = highest_energy_hour.value / total_megawatt_hours
    
    while peak_hours_percentage < 0.20 and left_index > -1 and right_index < len(demand_data):
        left_value = demand_data[left_index].value
        right_value = demand_data[right_index].value
        if left_value > right_value:
            peak_hours_percentage += (left_value / total_megawatt_hours)
            left_index -= 1
        else:
            peak_hours_percentage += (right_value / total_megawatt_hours)
            right_index += 1
    return demand_data[left_index], demand_data[right_index], peak_hours_percentage

find_peak_hour_timeframe(recent_demand_data)
